# Notebook 20: Text-to-Speech Inference

---

## Inference Engineering Course

Welcome to Notebook 20! In this final notebook, we explore **Text-to-Speech (TTS)** inference -- generating spoken audio from text using neural models.

### What You Will Learn

| Topic | Description |
|-------|-------------|
| **TTS Pipeline** | How text becomes speech |
| **Running a TTS Model** | Generate speech from text |
| **Time-to-First-Byte** | Measure latency metrics for streaming |
| **Audio Token Decoding** | Understanding the token-to-audio process |
| **Waveform Analysis** | Visualize generated waveforms and spectrograms |
| **Voice Parameters** | Compare different voices and speaking rates |

### Modern TTS Pipeline

```
Text -> Text Encoder -> Acoustic Model -> Audio Tokens -> Vocoder -> Waveform
```

Modern TTS systems (like Bark, VITS, or SpeechT5) often use neural vocoders and can produce remarkably natural-sounding speech.

---

## Part 1: Setup & Installations

In [ ]:
%%capture
!pip install transformers torch torchaudio datasets soundfile matplotlib numpy scipy

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
import soundfile as sf
from scipy import signal
from typing import Dict, List, Optional, Tuple
import IPython.display as ipd
warnings.filterwarnings('ignore')

plt.style.use('default')
plt.rcParams['figure.figsize'] = (14, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

## Part 2: Understanding the TTS Pipeline

### The Three Stages of TTS

| Stage | Input | Output | Purpose |
|-------|-------|--------|--------|
| **Text Processing** | Raw text | Phonemes/tokens | Normalize text, handle numbers, abbreviations |
| **Acoustic Model** | Phonemes | Mel spectrogram | Predict acoustic features from text |
| **Vocoder** | Mel spectrogram | Audio waveform | Generate actual sound waves |

### Key Metrics

- **Time-to-First-Byte (TTFB)**: How quickly the first audio chunk is ready
- **Real-Time Factor (RTF)**: Processing time / Audio duration (< 1 for real-time)
- **MOS (Mean Opinion Score)**: Human quality rating (1-5 scale)
- **Sample Rate**: Typically 16kHz-24kHz for TTS

In [ ]:
# Visualize the TTS pipeline stages
fig, ax = plt.subplots(figsize=(16, 4))

stages = [
    ('Input Text', '#E3F2FD', '"Hello, how are\nyou today?"'),
    ('Text\nProcessing', '#BBDEFB', 'Phonemes:\n/həˈloʊ/'),
    ('Acoustic\nModel', '#90CAF9', 'Mel\nSpectrogram'),
    ('Vocoder', '#64B5F6', 'Audio\nWaveform'),
    ('Output\nAudio', '#42A5F5', '16kHz WAV\n~2.5 seconds'),
]

for i, (name, color, detail) in enumerate(stages):
    rect = plt.Rectangle((i * 3, 0), 2.5, 2, facecolor=color, 
                          edgecolor='#1565C0', linewidth=2, zorder=2)
    ax.add_patch(rect)
    ax.text(i * 3 + 1.25, 1.5, name, ha='center', va='center', 
            fontsize=11, fontweight='bold', color='#0D47A1')
    ax.text(i * 3 + 1.25, 0.5, detail, ha='center', va='center', 
            fontsize=8, color='#1565C0')
    
    if i < len(stages) - 1:
        ax.annotate('', xy=(i * 3 + 3, 1), xytext=(i * 3 + 2.5, 1),
                    arrowprops=dict(arrowstyle='->', color='#1565C0', lw=2))

# Add timing annotations
timings = ['~1ms', '~50ms', '~200ms', '~100ms']
for i, t in enumerate(timings):
    ax.text(i * 3 + 2.75, -0.3, t, ha='center', fontsize=9, 
            color='#F44336', fontstyle='italic')

ax.set_xlim(-0.5, 15)
ax.set_ylim(-0.8, 2.5)
ax.axis('off')
ax.set_title('Text-to-Speech Pipeline', fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

## Part 3: Loading the TTS Model

We will use **SpeechT5** from Microsoft, available through HuggingFace Transformers. It is a versatile model that supports both TTS and ASR.

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset

print("Loading SpeechT5 TTS model...")

# Load processor (text tokenizer)
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

# Load TTS model (text -> mel spectrogram)
tts_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)

# Load vocoder (mel spectrogram -> waveform)
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(device)

# Load speaker embeddings
print("Loading speaker embeddings...")
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0).to(device)

tts_params = sum(p.numel() for p in tts_model.parameters())
voc_params = sum(p.numel() for p in vocoder.parameters())
print(f"\nTTS model parameters: {tts_params / 1e6:.1f}M")
print(f"Vocoder parameters: {voc_params / 1e6:.1f}M")
print(f"Total parameters: {(tts_params + voc_params) / 1e6:.1f}M")
print("Models loaded successfully!")

In [ ]:
@torch.no_grad()
def generate_speech(text: str, speaker_emb=None, 
                     measure_ttfb: bool = True) -> Dict:
    """
    Generate speech from text and measure performance metrics.
    
    Returns:
        Dict with audio waveform, mel spectrogram, and timing info
    """
    if speaker_emb is None:
        speaker_emb = speaker_embedding
    
    # Step 1: Text processing
    t_start = time.time()
    inputs = processor(text=text, return_tensors="pt").to(device)
    t_text = time.time() - t_start
    
    num_input_tokens = inputs['input_ids'].shape[1]
    
    # Step 2: Generate mel spectrogram (acoustic model)
    t_acoustic_start = time.time()
    spectrogram = tts_model.generate_speech(inputs["input_ids"], speaker_emb)
    t_acoustic = time.time() - t_acoustic_start
    
    # This is the "time to first byte" -- when the first audio frame is ready
    ttfb = t_text + t_acoustic
    
    # Step 3: Vocoder (mel -> waveform)
    t_vocoder_start = time.time()
    with torch.no_grad():
        waveform = vocoder(spectrogram)
    t_vocoder = time.time() - t_vocoder_start
    
    total_time = t_text + t_acoustic + t_vocoder
    
    # Convert to numpy
    audio_np = waveform.cpu().numpy().squeeze()
    mel_np = spectrogram.cpu().numpy().squeeze()
    
    # Calculate audio duration (16kHz sample rate)
    sample_rate = 16000
    audio_duration = len(audio_np) / sample_rate
    rtf = total_time / audio_duration if audio_duration > 0 else float('inf')
    
    return {
        'text': text,
        'audio': audio_np,
        'mel_spectrogram': mel_np,
        'sample_rate': sample_rate,
        'audio_duration_s': round(audio_duration, 3),
        'input_tokens': num_input_tokens,
        'mel_frames': mel_np.shape[0] if mel_np.ndim >= 1 else 0,
        'time_text_processing': round(t_text, 4),
        'time_acoustic_model': round(t_acoustic, 4),
        'time_vocoder': round(t_vocoder, 4),
        'time_total': round(total_time, 4),
        'ttfb': round(ttfb, 4),
        'rtf': round(rtf, 4),
    }

# Test generation
print("Generating test speech...")
result = generate_speech("Hello, welcome to the inference engineering course!")
print(f"\nGenerated speech:")
print(f"  Text: '{result['text']}'")
print(f"  Audio duration: {result['audio_duration_s']}s")
print(f"  Input tokens: {result['input_tokens']}")
print(f"  Mel frames: {result['mel_frames']}")
print(f"  Total time: {result['time_total']}s")
print(f"  TTFB: {result['ttfb']}s")
print(f"  RTF: {result['rtf']}")

# Play audio in notebook
ipd.display(ipd.Audio(result['audio'], rate=result['sample_rate']))

## Part 4: Visualizing Generated Waveforms and Spectrograms

Let's look at what the TTS model produces at each stage.

In [ ]:
def visualize_tts_output(result: Dict):
    """Comprehensive visualization of TTS output."""
    fig, axes = plt.subplots(3, 1, figsize=(16, 12))
    
    audio = result['audio']
    sr = result['sample_rate']
    mel = result['mel_spectrogram']
    
    # 1. Waveform
    ax = axes[0]
    time_axis = np.arange(len(audio)) / sr
    ax.plot(time_axis, audio, color='#2196F3', linewidth=0.5, alpha=0.8)
    ax.fill_between(time_axis, audio, alpha=0.1, color='#2196F3')
    ax.set_xlabel('Time (seconds)', fontsize=12)
    ax.set_ylabel('Amplitude', fontsize=12)
    ax.set_title(f'Generated Waveform: "{result["text"][:60]}..."',
                 fontsize=13, fontweight='bold')
    ax.set_xlim(0, len(audio) / sr)
    
    # 2. Mel Spectrogram (from acoustic model)
    ax = axes[1]
    if mel.ndim == 2:
        im = ax.imshow(mel.T, aspect='auto', origin='lower', cmap='magma')
        ax.set_ylabel('Mel Frequency Bin', fontsize=12)
    else:
        im = ax.imshow(mel.reshape(1, -1), aspect='auto', cmap='magma')
    ax.set_xlabel('Frame', fontsize=12)
    ax.set_title('Mel Spectrogram (Acoustic Model Output)', fontsize=13, fontweight='bold')
    plt.colorbar(im, ax=ax, label='Energy')
    
    # 3. Spectrogram computed from waveform
    ax = axes[2]
    frequencies, times, Sxx = signal.spectrogram(audio, sr, nperseg=512, noverlap=256)
    im = ax.pcolormesh(times, frequencies, 10 * np.log10(Sxx + 1e-10), 
                        shading='gouraud', cmap='magma')
    ax.set_ylabel('Frequency (Hz)', fontsize=12)
    ax.set_xlabel('Time (seconds)', fontsize=12)
    ax.set_title('Spectrogram (from Generated Audio)', fontsize=13, fontweight='bold')
    ax.set_ylim(0, 8000)  # Focus on speech frequencies
    plt.colorbar(im, ax=ax, label='Power (dB)')
    
    plt.tight_layout()
    plt.show()

# Visualize our test generation
visualize_tts_output(result)

## Part 5: Measuring Time-to-First-Byte (TTFB)

In streaming applications (like voice assistants), **TTFB** is crucial -- the user expects to hear audio quickly after submitting text.

TTFB depends on:
1. Text tokenization time (fast, ~1ms)
2. First mel frame generation time (depends on model)
3. Vocoder startup time

For non-streaming models (like SpeechT5), TTFB = total generation time. For streaming models, TTFB can be much lower.

In [ ]:
# Measure TTFB and total time for different text lengths
test_texts = [
    "Hi.",
    "Hello, how are you?",
    "The quick brown fox jumps over the lazy dog.",
    "In the world of artificial intelligence, inference engineering plays a crucial role in deploying models efficiently.",
    "Machine learning models require careful optimization for production deployment. This includes techniques like quantization, pruning, batching, and caching to achieve the best performance within hardware constraints.",
]

ttfb_results = []
print("Measuring TTFB and generation metrics...\n")

for text in test_texts:
    # Warmup
    _ = generate_speech(text)
    
    # Measure (average of 2 runs)
    runs = []
    for _ in range(2):
        r = generate_speech(text)
        runs.append(r)
    
    # Average the timing metrics
    avg_result = runs[0].copy()
    for key in ['time_text_processing', 'time_acoustic_model', 'time_vocoder', 
                'time_total', 'ttfb', 'rtf']:
        avg_result[key] = round(np.mean([r[key] for r in runs]), 4)
    
    ttfb_results.append(avg_result)
    
    text_preview = text[:50] + '...' if len(text) > 50 else text
    print(f"Text ({avg_result['input_tokens']:>3d} tokens): \"{text_preview}\"")
    print(f"  Audio: {avg_result['audio_duration_s']}s | Total: {avg_result['time_total']}s | TTFB: {avg_result['ttfb']}s | RTF: {avg_result['rtf']}")
    print(f"  Breakdown: text={avg_result['time_text_processing']}s, acoustic={avg_result['time_acoustic_model']}s, vocoder={avg_result['time_vocoder']}s")
    print()

In [ ]:
# Visualize TTFB and timing breakdown
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

input_tokens = [r['input_tokens'] for r in ttfb_results]

# Plot 1: Time breakdown stacked bar
ax = axes[0]
x = np.arange(len(ttfb_results))
t_text = [r['time_text_processing'] for r in ttfb_results]
t_acoustic = [r['time_acoustic_model'] for r in ttfb_results]
t_vocoder = [r['time_vocoder'] for r in ttfb_results]

ax.bar(x, t_text, label='Text Processing', color='#4CAF50', alpha=0.8)
ax.bar(x, t_acoustic, bottom=t_text, label='Acoustic Model', color='#2196F3', alpha=0.8)
ax.bar(x, t_vocoder, bottom=[a+b for a,b in zip(t_text, t_acoustic)], 
       label='Vocoder', color='#FF9800', alpha=0.8)

ax.set_xticks(x)
ax.set_xticklabels([f'{t} tok' for t in input_tokens], fontsize=9)
ax.set_xlabel('Input Length (tokens)', fontsize=12)
ax.set_ylabel('Time (seconds)', fontsize=12)
ax.set_title('Time Breakdown by Stage', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)

# Plot 2: TTFB vs input length
ax = axes[1]
ttfbs = [r['ttfb'] for r in ttfb_results]
ax.plot(input_tokens, ttfbs, 'o-', color='#F44336', linewidth=2.5, markersize=10)
ax.fill_between(input_tokens, ttfbs, alpha=0.1, color='#F44336')

# Add acceptable TTFB threshold
ax.axhline(y=0.5, color='green', linestyle='--', alpha=0.5, label='500ms threshold')
ax.axhline(y=1.0, color='orange', linestyle='--', alpha=0.5, label='1s threshold')

ax.set_xlabel('Input Tokens', fontsize=12)
ax.set_ylabel('TTFB (seconds)', fontsize=12)
ax.set_title('Time-to-First-Byte', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)

# Plot 3: RTF vs input length
ax = axes[2]
rtfs = [r['rtf'] for r in ttfb_results]
colors = ['#4CAF50' if rtf < 1 else '#F44336' for rtf in rtfs]
bars = ax.bar(x, rtfs, color=colors, alpha=0.8, edgecolor='black')
ax.axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='Real-time')

ax.set_xticks(x)
ax.set_xticklabels([f'{t} tok' for t in input_tokens], fontsize=9)
ax.set_xlabel('Input Length (tokens)', fontsize=12)
ax.set_ylabel('Real-Time Factor', fontsize=12)
ax.set_title('RTF (lower = faster)', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)

for bar, rtf in zip(bars, rtfs):
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.02,
           f'{rtf:.3f}', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

## Part 6: Audio Token Decoding Process

TTS models generate audio through a multi-step decoding process. Let's visualize how the mel spectrogram is built up over time.

In [ ]:
# Simulate the progressive mel spectrogram generation
text = "The future of AI is incredibly exciting."
full_result = generate_speech(text)
full_mel = full_result['mel_spectrogram']

if full_mel.ndim == 2:
    num_frames = full_mel.shape[0]
    num_bins = full_mel.shape[1]
    
    # Show progressive generation
    percentages = [10, 25, 50, 75, 100]
    fig, axes = plt.subplots(1, len(percentages), figsize=(4 * len(percentages), 4))
    
    for ax, pct in zip(axes, percentages):
        frames_to_show = int(num_frames * pct / 100)
        partial_mel = np.zeros_like(full_mel)
        partial_mel[:frames_to_show] = full_mel[:frames_to_show]
        
        ax.imshow(partial_mel.T, aspect='auto', origin='lower', cmap='magma',
                  vmin=full_mel.min(), vmax=full_mel.max())
        ax.axvline(x=frames_to_show, color='lime', linestyle='--', linewidth=2)
        ax.set_title(f'{pct}% Generated\n({frames_to_show}/{num_frames} frames)',
                     fontsize=11, fontweight='bold')
        ax.set_xlabel('Frame')
        if pct == 10:
            ax.set_ylabel('Mel Bin')
    
    plt.suptitle('Progressive Mel Spectrogram Generation',
                 fontsize=15, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    print(f"Total mel frames: {num_frames}")
    print(f"Mel bins: {num_bins}")
    print(f"Audio samples: {len(full_result['audio'])}")
    print(f"Mel-to-audio ratio: ~{len(full_result['audio']) // num_frames} samples per frame")
else:
    print(f"Mel spectrogram shape: {full_mel.shape}")
    print("Visualization adapted for 1D mel output.")

In [ ]:
# Visualize the vocoder's role: mel -> waveform conversion
fig, axes = plt.subplots(2, 1, figsize=(16, 8))

mel = full_result['mel_spectrogram']
audio = full_result['audio']
sr = full_result['sample_rate']

# Top: Mel spectrogram
ax = axes[0]
if mel.ndim == 2:
    extent = [0, len(audio)/sr, 0, mel.shape[1]]
    ax.imshow(mel.T, aspect='auto', origin='lower', cmap='magma', extent=extent)
ax.set_ylabel('Mel Frequency Bin', fontsize=12)
ax.set_title('Acoustic Model Output (Mel Spectrogram)', fontsize=14, fontweight='bold')

# Bottom: Waveform
ax = axes[1]
time_axis = np.arange(len(audio)) / sr
ax.plot(time_axis, audio, color='#2196F3', linewidth=0.3, alpha=0.8)

# Add envelope
window = int(sr * 0.02)  # 20ms window
if len(audio) > window:
    envelope = np.array([np.max(np.abs(audio[max(0,i-window):i+window])) 
                          for i in range(0, len(audio), window // 2)])
    env_time = np.linspace(0, len(audio)/sr, len(envelope))
    ax.plot(env_time, envelope, color='#F44336', linewidth=2, alpha=0.7, label='Envelope')
    ax.plot(env_time, -envelope, color='#F44336', linewidth=2, alpha=0.7)

ax.set_xlabel('Time (seconds)', fontsize=12)
ax.set_ylabel('Amplitude', fontsize=12)
ax.set_title('Vocoder Output (Audio Waveform)', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.set_xlim(0, len(audio)/sr)

# Draw connection arrows
fig.text(0.02, 0.5, 'Vocoder\nConverts', ha='center', va='center',
         fontsize=11, fontweight='bold', color='#4CAF50',
         rotation=90)

plt.tight_layout()
plt.show()

## Part 7: Comparing Different Voice Parameters

SpeechT5 uses speaker embeddings to control the voice. Let's compare different speakers.

In [ ]:
# Compare different speaker embeddings
text = "The quick brown fox jumps over the lazy dog."

# Select different speaker embeddings from the dataset
speaker_indices = [7306, 2000, 5000, 1000, 8000]
speaker_results = []

print(f"Generating speech with {len(speaker_indices)} different speakers...\n")

for idx in speaker_indices:
    try:
        spk_emb = torch.tensor(embeddings_dataset[idx]["xvector"]).unsqueeze(0).to(device)
        result = generate_speech(text, speaker_emb=spk_emb)
        result['speaker_idx'] = idx
        speaker_results.append(result)
        print(f"Speaker {idx}: duration={result['audio_duration_s']}s, RTF={result['rtf']}")
    except Exception as e:
        print(f"Speaker {idx}: Error - {e}")

# Visualize waveform comparison
fig, axes = plt.subplots(len(speaker_results), 1, figsize=(16, 3 * len(speaker_results)))
if len(speaker_results) == 1:
    axes = [axes]

colors = ['#2196F3', '#4CAF50', '#FF9800', '#F44336', '#9C27B0']

for i, (ax, result) in enumerate(zip(axes, speaker_results)):
    audio = result['audio']
    sr = result['sample_rate']
    t = np.arange(len(audio)) / sr
    
    ax.plot(t, audio, color=colors[i % len(colors)], linewidth=0.5, alpha=0.8)
    ax.set_title(f'Speaker {result["speaker_idx"]} | Duration: {result["audio_duration_s"]}s',
                 fontsize=12, fontweight='bold')
    ax.set_ylabel('Amplitude')
    ax.set_xlim(0, max(r['audio_duration_s'] for r in speaker_results))

axes[-1].set_xlabel('Time (seconds)', fontsize=12)

plt.suptitle(f'Same Text, Different Speakers: "{text}"',
             fontsize=15, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

# Play each speaker's audio
for result in speaker_results:
    print(f"\nSpeaker {result['speaker_idx']}:")
    ipd.display(ipd.Audio(result['audio'], rate=result['sample_rate']))

In [ ]:
# Analyze spectral differences between speakers
fig, axes = plt.subplots(1, min(3, len(speaker_results)), 
                          figsize=(6 * min(3, len(speaker_results)), 4))
if not isinstance(axes, np.ndarray):
    axes = [axes]

for i, (ax, result) in enumerate(zip(axes, speaker_results[:3])):
    audio = result['audio']
    sr = result['sample_rate']
    
    # Compute spectrogram
    f, t, Sxx = signal.spectrogram(audio, sr, nperseg=512, noverlap=256)
    ax.pcolormesh(t, f, 10 * np.log10(Sxx + 1e-10), shading='gouraud', cmap='viridis')
    ax.set_ylim(0, 8000)
    ax.set_xlabel('Time (s)', fontsize=11)
    ax.set_ylabel('Frequency (Hz)', fontsize=11)
    ax.set_title(f'Speaker {result["speaker_idx"]}', fontsize=12, fontweight='bold')

plt.suptitle('Spectral Comparison Across Speakers',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## Part 8: TTS Performance Analysis

Let's create a comprehensive performance analysis of the TTS system.

In [ ]:
# Comprehensive performance analysis
perf_texts = [
    "Hi.",
    "Hello world.",
    "This is a short sentence.",
    "The quick brown fox jumps over the lazy dog.",
    "In the beginning, there was nothing but the vast emptiness of space.",
    "Machine learning models are becoming increasingly important for a wide range of applications in industry and research.",
]

perf_results = []
print("Running performance analysis...")
for text in perf_texts:
    # Average of 2 runs
    runs = [generate_speech(text) for _ in range(2)]
    avg = runs[0].copy()
    for key in ['time_total', 'ttfb', 'rtf', 'time_acoustic_model', 'time_vocoder']:
        avg[key] = round(np.mean([r[key] for r in runs]), 4)
    perf_results.append(avg)
    print(f"  {avg['input_tokens']:>3d} tokens -> {avg['audio_duration_s']}s audio ({avg['time_total']}s)")

# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

tokens = [r['input_tokens'] for r in perf_results]
audio_durs = [r['audio_duration_s'] for r in perf_results]
total_times = [r['time_total'] for r in perf_results]
ttfbs = [r['ttfb'] for r in perf_results]
rtfs = [r['rtf'] for r in perf_results]

# Plot 1: Input tokens vs output audio duration
ax = axes[0][0]
ax.plot(tokens, audio_durs, 'o-', color='#2196F3', linewidth=2.5, markersize=10)
ax.set_xlabel('Input Tokens', fontsize=12)
ax.set_ylabel('Audio Duration (seconds)', fontsize=12)
ax.set_title('Output Duration vs Input Length', fontsize=14, fontweight='bold')

# Plot 2: Generation time breakdown
ax = axes[0][1]
t_acoustic = [r['time_acoustic_model'] for r in perf_results]
t_vocoder = [r['time_vocoder'] for r in perf_results]
ax.stackplot(tokens, t_acoustic, t_vocoder,
             labels=['Acoustic Model', 'Vocoder'],
             colors=['#2196F3', '#FF9800'], alpha=0.8)
ax.set_xlabel('Input Tokens', fontsize=12)
ax.set_ylabel('Time (seconds)', fontsize=12)
ax.set_title('Generation Time Breakdown', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)

# Plot 3: RTF trend
ax = axes[1][0]
colors_rtf = ['#4CAF50' if r < 1 else '#F44336' for r in rtfs]
ax.bar(range(len(rtfs)), rtfs, color=colors_rtf, alpha=0.8, edgecolor='black')
ax.axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='Real-time')
ax.set_xticks(range(len(tokens)))
ax.set_xticklabels([f'{t}' for t in tokens])
ax.set_xlabel('Input Tokens', fontsize=12)
ax.set_ylabel('RTF', fontsize=12)
ax.set_title('Real-Time Factor by Input Length', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)

# Plot 4: Tokens per second of audio
ax = axes[1][1]
tok_per_audio_s = [t / d if d > 0 else 0 for t, d in zip(tokens, audio_durs)]
ax.plot(tokens, tok_per_audio_s, 's-', color='#9C27B0', linewidth=2.5, markersize=10)
ax.set_xlabel('Input Tokens', fontsize=12)
ax.set_ylabel('Tokens per Second of Audio', fontsize=12)
ax.set_title('Text Density in Output Audio', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## Part 9: Key Takeaways

### Summary

1. **TTS is a multi-stage pipeline**: Text processing, acoustic model, and vocoder each contribute to total latency.

2. **Time-to-First-Byte (TTFB)** is the critical metric for interactive applications. Non-streaming models have TTFB = total time; streaming models can achieve much lower TTFB.

3. **Real-Time Factor (RTF)** determines if the system can be used for live/streaming applications. RTF < 1 is required.

4. **Speaker embeddings** control voice characteristics without retraining the model -- a powerful inference-time control.

5. **The vocoder** is often the bottleneck. HiFi-GAN is fast but models like WaveNet are higher quality but much slower.

### Production TTS Optimization

| Optimization | Expected Speedup | Notes |
|-------------|-----------------|-------|
| FP16 inference | 1.5-2x | Minimal quality impact |
| Streaming/chunked | Lower TTFB | Start playing before full generation |
| TensorRT vocoder | 2-3x | Significant for vocoder-bound systems |
| Smaller model | 2-5x | Quality tradeoff |
| Caching | Variable | Cache common phrases |

### The TTS Landscape (2024-2025)

| Model | Quality | Speed | Open Source |
|-------|---------|-------|-------------|
| SpeechT5 | Good | Fast | Yes |
| Bark | Very Good | Slow | Yes |
| VITS | Good | Fast | Yes |
| Tortoise TTS | Excellent | Very Slow | Yes |
| ElevenLabs | Excellent | Fast | No (API) |

---

## Exercises

### Exercise 1: Batch TTS Processing
Measure the throughput when generating speech for multiple sentences.

In [ ]:
# Exercise 1: Batch TTS throughput
# Generate speech for a list of 10 sentences
# Measure: total audio generated per second of compute

batch_texts = [
    "Welcome to the conference.",
    "Please take your seats.",
    "The keynote will begin shortly.",
    "Thank you for joining us today.",
    "Let me introduce our first speaker.",
]

# TODO: Process all sentences, measure total time, calculate throughput
# Throughput = total audio seconds generated / total compute seconds

print("Exercise 1: Measure batch TTS throughput!")

### Exercise 2: Speaker Interpolation
Try interpolating between two speaker embeddings to create a blended voice.

In [ ]:
# Exercise 2: Speaker interpolation
# Take two speaker embeddings and blend them:
# blended = alpha * speaker_1 + (1 - alpha) * speaker_2
# Try alpha = 0.0, 0.25, 0.5, 0.75, 1.0
# Listen to how the voice changes

# TODO: Implement speaker interpolation

print("Exercise 2: Experiment with speaker interpolation!")

### Exercise 3: Streaming Simulation
Simulate a streaming TTS system by generating audio in chunks and measuring per-chunk latency.

In [ ]:
# Exercise 3: Simulate streaming TTS
# Split a long text into sentences
# Generate audio for each sentence sequentially
# Measure the "streaming TTFB" (time until first sentence is ready)
# Compare with generating all at once

long_text = (
    "Welcome to the world of speech synthesis. "
    "Today we will explore how machines can talk. "
    "This technology has improved dramatically in recent years. "
    "Modern systems sound remarkably natural."
)

# TODO: Implement sentence-by-sentence streaming simulation

print("Exercise 3: Simulate streaming TTS!")

---

**End of Notebook 20: Text-to-Speech Inference**

---

## Congratulations!

You have completed the Inference Engineering notebook series (Notebooks 14-20)! Here is a summary of what we covered:

| Notebook | Topic | Key Concept |
|----------|-------|-------------|
| 14 | N-gram Speculation | Predict tokens from n-gram patterns |
| 15 | Speculative Decoding | Draft model + target model verification |
| 16 | Embedding Models | Vector similarity for semantic search |
| 17 | Image Generation | Guidance scale and step count tradeoffs |
| 18 | VLM Inference | Processing images as visual tokens |
| 19 | ASR with Whisper | Audio-to-text with real-time constraints |
| 20 | TTS Inference | Text-to-audio with latency optimization |

Each of these represents a key modality in modern AI inference engineering. The techniques you have learned -- from speculative decoding to RTF measurement to TTFB analysis -- form the foundation of production AI system optimization.